In [1]:
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [4]:
n_vocab = 6785
n_embed = 300
n_filters = 250
n_classes = 2

In [5]:
class Model(nn.Module):
    '''Deep Pyramid Convolutional Neural Networks for Text Categorization'''
    def __init__(self, n_vocab, n_embed, n_filters, n_classes):
        super(Model, self).__init__()
        
        self.embedding = nn.Embedding(n_vocab, n_embed, padding_idx=n_vocab - 1)
        self.conv_region = nn.Conv2d(1, n_filters, (3, n_embed), stride=1)
        self.conv = nn.Conv2d(n_filters, n_filters, (3, 1), stride=1)
        self.max_pool = nn.MaxPool2d(kernel_size=(3, 1), stride=2)
        self.padding1 = nn.ZeroPad2d((0, 0, 1, 1))  # top bottom
        self.padding2 = nn.ZeroPad2d((0, 0, 0, 1))  # bottom
        self.relu = nn.ReLU()
        self.fc = nn.Linear(n_filters, n_classes)

    def forward(self, x):
        x = x[0]
        x = self.embedding(x)
        x = x.unsqueeze(1)         # [batch_size, 250, seq_len, 1]
        # x = self.conv_region(x)  # [batch_size, 250, seq_len-3+1, 1]
        px = self.conv_region(x)
        x = self.padding1(px)
        
        # x = self.padding1(x)     # [batch_size, 250, seq_len, 1]
        x = self.relu(x)
        x = self.conv(x)           # [batch_size, 250, seq_len-3+1, 1]
        x = self.padding1(x)       # [batch_size, 250, seq_len, 1]
        x = self.relu(x)
        x = self.conv(x)           # [batch_size, 250, seq_len-3+1, 1]
        print("0----", x.size())
        x = x + px
        while x.size()[2] >= 2:
            x = self._block(x)
            print(x.size())
        print("1----", x.size())
        x = x.squeeze()          # [batch_size, n_filters(250)]
        print("2----", x.size())
        x = self.fc(x)
        print("3----", x.size())
        return x

    def _block(self, x):
        x = self.padding2(x)
        px = self.max_pool(x)

        x = self.padding1(px)
        x = F.relu(x)
        x = self.conv(x)

        x = self.padding1(x)
        x = F.relu(x)
        x = self.conv(x)

        x = x + px
        return x

In [8]:
model = Model(n_vocab, n_embed, n_filters, n_classes)

In [53]:
model

Model(
  (embedding): Embedding(6785, 300, padding_idx=6784)
  (conv_region): Conv2d(1, 250, kernel_size=(3, 300), stride=(1, 1))
  (conv): Conv2d(250, 250, kernel_size=(3, 1), stride=(1, 1))
  (max_pool): MaxPool2d(kernel_size=(3, 1), stride=2, padding=0, dilation=1, ceil_mode=False)
  (padding1): ZeroPad2d(padding=(0, 0, 1, 1), value=0.0)
  (padding2): ZeroPad2d(padding=(0, 0, 0, 1), value=0.0)
  (relu): ReLU()
  (fc): Linear(in_features=250, out_features=2, bias=True)
)

In [9]:
#(x, seq_len), y
# x = torch.LongTensor([_[0] for _ in datas]).to(self.device)
# y = torch.LongTensor([_[1] for _ in datas]).to(self.device)
# # pad前的长度(超过pad_size的设为pad_size)
# seq_len = torch.LongTensor([_[2] for _ in datas]).to(self.device)
# return (x, seq_len), y
pad_size = 1500

In [10]:
data = []
for i in range(128):
    x = torch.randint(0, n_vocab, (pad_size,))
    y = 1
    seq_len = pad_size
    data.append((x, y, seq_len))

In [11]:
x = torch.LongTensor([_[0].numpy() for _ in data])
y = torch.LongTensor([_[1] for _ in data])
seq_len = torch.LongTensor([_[2] for _ in data])
batch = (x, seq_len), y

In [12]:
new_data = (x, seq_len), y

In [13]:
new_data

((tensor([[1342, 5905, 6174,  ..., 2263, 6426,   36],
          [1689, 2948, 3520,  ...,  765, 3479, 2672],
          [2407, 2731, 4173,  ..., 2783, 1293, 2736],
          ...,
          [4387, 2260, 4780,  ..., 4688, 1345, 5652],
          [4265, 4354, 3805,  ...,  691,  548, 4485],
          [ 349, 3493, 2917,  ..., 2648, 3049, 4463]]),
  tensor([1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
          1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
          1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
          1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
          1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
          1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
          1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
          1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500,
   

In [14]:
model(new_data[0])
# 1---- torch.Size([128, 250, 2, 1])
# 2---- torch.Size([128, 250, 2])
## 正常
# 1---- torch.Size([128, 250, 1, 1])
# 2---- torch.Size([128, 250])
# 3---- torch.Size([128, 2])

0---- torch.Size([128, 250, 1498, 1])
torch.Size([128, 250, 749, 1])
torch.Size([128, 250, 374, 1])
torch.Size([128, 250, 187, 1])
torch.Size([128, 250, 93, 1])
torch.Size([128, 250, 46, 1])
torch.Size([128, 250, 23, 1])
torch.Size([128, 250, 11, 1])
torch.Size([128, 250, 5, 1])
torch.Size([128, 250, 2, 1])
torch.Size([128, 250, 1, 1])
1---- torch.Size([128, 250, 1, 1])
2---- torch.Size([128, 250])
3---- torch.Size([128, 2])


tensor([[ 5.3281, -2.7777],
        [ 5.2320, -2.7090],
        [ 5.5482, -2.8612],
        [ 5.2416, -2.9560],
        [ 5.2208, -2.7343],
        [ 5.1564, -2.5890],
        [ 5.5329, -2.5668],
        [ 5.4914, -2.9305],
        [ 5.5914, -2.8574],
        [ 5.2369, -3.0728],
        [ 5.3913, -2.6860],
        [ 5.2165, -2.7530],
        [ 5.5572, -2.8770],
        [ 5.5072, -2.7613],
        [ 5.1080, -2.7080],
        [ 5.2611, -2.9025],
        [ 5.4379, -2.7826],
        [ 5.4010, -2.9060],
        [ 5.3196, -2.6693],
        [ 5.3759, -2.8829],
        [ 5.3201, -2.7791],
        [ 5.3501, -2.6059],
        [ 5.5397, -2.7856],
        [ 5.2804, -2.9361],
        [ 4.9957, -2.9699],
        [ 5.1779, -2.9610],
        [ 5.5346, -2.7787],
        [ 5.6908, -3.2362],
        [ 5.2998, -2.7599],
        [ 5.6938, -2.8833],
        [ 5.3362, -2.8757],
        [ 4.9441, -2.7104],
        [ 5.3675, -3.0572],
        [ 5.1318, -2.8221],
        [ 5.4408, -2.9012],
        [ 5.6716, -2

In [15]:
new_data[0][0].size()

torch.Size([128, 1500])

In [16]:
nn.ZeroPad2d((0, 0, 1, 1))

ZeroPad2d(padding=(0, 0, 1, 1), value=0.0)

In [17]:
embedding = nn.Embedding(n_vocab, n_embed, padding_idx=n_vocab - 1)

In [18]:
x = new_data[0][0]

In [19]:
x.size()

torch.Size([128, 1500])

In [20]:
x = embedding(x)
x.size()

torch.Size([128, 1500, 300])

In [22]:
region_conv = nn.Conv2d(1, n_filters, (3, n_embed), stride=1)

In [23]:
x = x.unsqueeze(1)
x.size()

torch.Size([128, 1, 1500, 300])

In [24]:
x = region_conv(x)
x.size()

torch.Size([128, 250, 1498, 1])

In [25]:
padding1 = nn.ZeroPad2d((0, 0, 1, 1))

In [26]:
x = padding1(x)
x.size()

torch.Size([128, 250, 1500, 1])

In [27]:
conv = nn.Conv2d(n_filters, n_filters, (3, 1), stride=1)

In [28]:
relu = nn.ReLU()

In [29]:
x = relu(x)
print(x.size())
x = conv(x)
print(x.size())

torch.Size([128, 250, 1500, 1])
torch.Size([128, 250, 1498, 1])


In [30]:
x = padding1(x)
print(x.size())
x = relu(x)
print(x.size())
x = conv(x)
print(x.size())

torch.Size([128, 250, 1500, 1])
torch.Size([128, 250, 1500, 1])
torch.Size([128, 250, 1498, 1])


In [31]:
padding2 = nn.ZeroPad2d((0, 0, 0, 1))

In [32]:
maxpool = nn.MaxPool2d(kernel_size=(3, 1), stride=2)

In [33]:
x = padding2(x)

In [34]:
print(x.size())

torch.Size([128, 250, 1499, 1])


In [35]:
px = maxpool(x)
print(px.size())

torch.Size([128, 250, 749, 1])


In [36]:
x = padding1(px)
print(x.size())
x = relu(x)
print(x.size())
x = conv(x)
print(x.size())

torch.Size([128, 250, 751, 1])
torch.Size([128, 250, 751, 1])
torch.Size([128, 250, 749, 1])


In [37]:
x = padding1(x)
print(x.size())
x = relu(x)
print(x.size())
x = conv(x)
print(x.size())

torch.Size([128, 250, 751, 1])
torch.Size([128, 250, 751, 1])
torch.Size([128, 250, 749, 1])


In [38]:
x = x + px
x.size()

torch.Size([128, 250, 749, 1])